# 01. Market Expectations In The Cross-Section Of Present Values

TBD

In [ ]:
import numpy as np
import pandas as pd
import sys

from matplotlib import pyplot as plt
from settings import config
from pathlib import Path
import load_data
import regressions
import pull_CRSP_index
import pull_ken_french_data
import pull_fred

# Load environment variables
DATA_DIR = Path(config("DATA_DIR"))
OUTPUT_DIR = Path(config("OUTPUT_DIR"))
WRDS_USERNAME = config("WRDS_USERNAME")
START_DATE = config("START_DATE")
END_DATE = config("END_DATE")

## Prep Data

### Step 1: Fama-French Data

In [ ]:
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web

get_available_datasets()

In [ ]:
bm_6_portfolios = web.DataReader("6_Portfolios_2x3", "famafrench")

In [ ]:
print(bm_6_portfolios['DESCR'])

In [ ]:
bm_6_portfolios[6].head()

### Step 2: CRSP Market Capitalization Weighted Index Data

In [ ]:
crsp = pull_CRSP_index.pull_crsp_value_weighted_index()

In [ ]:
crsp_loaded = load_data.load_crsp_index()
crsp_loaded

### Step 3: Risk-Free Rate Data

In [ ]:
rf = pull_fred.pull_fred()
rf

In [ ]:
rf_monthly = load_data.load_fred_data()
rf_monthly

### Step 4: Excess Returns

In [ ]:
ex_ret = load_data.load_and_compute_excess_returns()
ex_ret

# Run Regressions

In [ ]:
import importlib
importlib.reload(load_data)

In [ ]:
# def regress():
#     portfolio_datasets = {
#         "6-Portfolios": "6_Portfolios_2x3",
#         "25-Portfolios": "25_Portfolios_5x5",
#         "100-Portfolios": "100_Portfolios_10x10"
#     }
#     results = {}
#     for label, dataset_name in portfolio_datasets.items():
#         print(f"\nProcessing {label} Data")
#         in_sample = regressions.run_in_sample_pls(dataset_name, weighting="value-weighted", h=1)
#         recursive = regressions.run_recursive_forecast(dataset_name, weighting="value-weighted", h=1)
#         regressions.display_results(label, in_sample, recursive)
#         results[label] = {"in_sample": in_sample, "recursive": recursive}
#     return results

# regress()

# Regressions for B/M ratios

In [ ]:
def regress():
    portfolio_datasets = {
        # "6-Portfolios": "6_Portfolios_2x3",
        # "25-Portfolios": "25_Portfolios_5x5",
        "100-Portfolios": "100_Portfolios_10x10"
    }
    results = {}
    for label, dataset_name in portfolio_datasets.items():
        print(f"\nProcessing {label} Data")
        in_sample = regressions.run_in_sample_pls(
            dataset_name=dataset_name, 
            weighting="BE_FYt-1_to_ME_June_t", 
            h=1, 
            end_date='1980-01-01')
        recursive = regressions.run_recursive_forecast(
            dataset_name=dataset_name, 
            weighting="BE_FYt-1_to_ME_June_t", 
            h=1, 
            start_train_date='1934-02-01', 
            end_train_date='1980-01-01',
            end_forecast_date='2011-01-01')
        regressions.display_results(label, in_sample, recursive)
        results[label] = {"in_sample": in_sample, "recursive": recursive}
    return results

regress()